**IMPORTS**

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import spacy
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

from nltk.stem import WordNetLemmatizer
import spacy

# Plotting tools
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, f1_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB, BernoulliNB, MultinomialNB
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)
# Input data files are available in the "../input/" directory.

import os
n = 1000

In [ ]:
!pip install  tensorflow-gpu==1.15.0

In [ ]:
!unzip cased_L-12_H-768_A-12.zip

In [ ]:
import tensorflow as tf

In [ ]:
import bert
from bert import run_classifier
from bert import optimization
from bert import tokenization
from bert import modeling
import tensorflow as tf
import numpy as np
import itertools

# NLTK Stop words

In [ ]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['background', 'methods', 'introduction', 'conclusions', 'results', 
                   'purpose', 'materials', 'discussions','methodology','result analysis'])

list of files given for [COVID-19 Open Research Dataset challenge](https://www.kaggle.com/allen-institute-for-ai/CORD-19-research-challenge) 

In [ ]:
os.listdir('../input/CORD-19-research-challenge/')

* read the readme file first

In [ ]:
with open('../input/CORD-19-research-challenge/metadata.readme', 'r') as f:
    data = f.read()
    print(data)

In [ ]:
biorxiv_dir = '/kaggle/input/CORD-19-research-challenge/document_parses/pdf_json'
filenames = os.listdir(biorxiv_dir)
print("Number of articles retrieved from biorxiv:", len(filenames))

In [ ]:
#The cell below shows that the updated datasets are in CSV format now - biorxiv_clean.csv, 
# clean_comm_use.csv, clean_noncomm_use.csv and clean_pmc.csv
os.listdir( '/kaggle/input/cord-19-eda-parse-json-and-generate-clean-csv')

**Reading The updated clean CSV files**
### Show the first 2 rows of each sub CSV files of the dataset bu using name of csv file.head(2)

In [ ]:
biorxiv_clean = pd.read_csv('/kaggle/input/cord-19-eda-parse-json-and-generate-clean-csv/biorxiv_clean.csv')
clean_comm_use = pd.read_csv('/kaggle/input/cord-19-eda-parse-json-and-generate-clean-csv/clean_comm_use.csv')
clean_noncomm_use =  pd.read_csv('/kaggle/input/cord-19-eda-parse-json-and-generate-clean-csv/clean_noncomm_use.csv')
clean_pmc =  pd.read_csv('/kaggle/input/cord-19-eda-parse-json-and-generate-clean-csv/clean_pmc.csv')

In [ ]:
biorxiv_clean.head(2)

In [ ]:
clean_comm_use.head(2)

In [ ]:
clean_noncomm_use.head(2)

In [ ]:
clean_pmc.head(2)

first text of biorxiv_clean dataset

In [ ]:
biorxiv_clean.text[0]

# Part 1 :  Working with biorxiv

# biorxiv_clean papers Abstract - frequent words (400 sample)

In [ ]:
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook

stopwords = set(STOPWORDS)

def show_wordcloud(data, title = None):
    wordcloud = WordCloud(
        background_color='white',
        stopwords=stopwords,
        max_words=200,
        max_font_size=30, 
        scale=5,
        random_state=1
    ).generate(str(data))

    fig = plt.figure(1, figsize=(10,10))
    plt.axis('off')
    if title: 
        fig.suptitle(title, fontsize=14)
        fig.subplots_adjust(top=2.3)

    plt.imshow(wordcloud)
    plt.show()

In [ ]:
show_wordcloud(biorxiv_clean['abstract'], title = 'biorxiv_clean - papers Abstract - frequent words (400 sample)')

# Convert abstract to list

In [ ]:
df = biorxiv_clean
df = df.abstract.dropna() 
data = df.values.tolist()

dropna() will delete rows that have missing value in the abstract column, tolist() converts the dataframe into a list.

* all the sentences and corresponding similar sentences have been stored in a pandas dataframe called simsentence**

### use the new dataframe as csv file for possible further research

In [ ]:
simsentence = pd.read_csv('../input/simsentence/simsentence.csv')

**The function below converts sentences to words using gensim**

In [ ]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  

data_words = list(sent_to_words(data))

print(data_words[:1])

yield will return a generator, that used gensim.utils.simple_preprocess() function and for each sentence, will convert it into list of preprocessed words

# Build the bigram and trigram models using gensim

In [ ]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=20) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=20)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[1]]])

creating a bigram model using the Phrases class from Gensim, using data_words which contains preprocessed word lists. The min_count parameter specifies the minimum frequency a phrase needs to appear in the corpus to be considered as a bigram, and the threshold parameter controls the threshold for forming the bigrams.

The Phraser class is a faster way to apply a trained phrase model to a sequence.


# Define functions for stopwords, bigrams, trigrams and lemmatization

In [ ]:
#https://github.com/cjriggio/classifying_medical_innovation
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out


text processing and lemmatization.

remove_stopwords(texts), make_bigrams(texts), make_trigrams(texts) all are use a list comprehension to iterate over each word in texts lists. 

then, performs lemmatization on a list of texts using SpaCy. It takes the texts list as input and Within the loop, it joins the words in each sentence with a space and processes them using SpaCy's nlp pipeline.

It then retrieves the lemma of each token in the sentence token.lemma_ 
if its part-of-speech token.pos_ 


In [ ]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[1])




nlp = spacy.load('en', disable=['parser', 'ner']):  By default, when loading the English model in SpaCy, both the tagger and the named entity recognizer components are enabled.but here we load the SpaCy English model with only the tagger component enabled. 

The allowed_postags argument allows to specify the part-of-speech tags for which lemmatization should be applied. In this case, the allowed tags are 'NOUN', 'ADJ', 'VERB', and 'ADV', which correspond to nouns, adjectives, verbs, and adverbs, respectively.

When the lemmatization() function is called, it will iterate over the tokens in each sentence and check their part-of-speech tags. Only the tokens with allowed tags will be lemmatized, and the rest will be excluded.


In [ ]:
print(data_lemmatized[:1])


# Create Dictionary,Corpus and Document Frequency

In [ ]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

id2word is a Dictionary object that maps words to unique IDs created using the corpora.Dictionary class from Gensim.

The doc2bow() method performs the following steps:

Tokenization: It tokenizes the input text, splitting it into individual words or tokens.

Word ID mapping: For each token in the text, it checks if the token exists in the dictionary (id2word). If the token is present, it retrieves the corresponding unique ID assigned to that word. If the token is not present, it assigns a new unique ID to the token and adds it to the dictionary.

Counting word occurrences: It counts the frequency of each word in the document.

Output: The method returns a list of (word_id, word_frequency) tuples, where word_id is the unique ID of the word in the dictionary, and word_frequency is the count of occurrences of that word in the document.

The resulting list of tuples represents the bag-of-words representation of the document, where each word is represented by its ID and frequency in the document.

# Human readable format of corpus (term-frequency)

In [ ]:
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

# Part 2 : Working with clean_comm_use.csv

In [ ]:
df1 = clean_comm_use
df1 = df1.dropna()

# creating n-gram and fetching to HashingVectorizer to get feature vector X

In [ ]:
#https://www.kaggle.com/maksimeren/covid-19-literature-clustering
words = []
for ii in range(0,len(df1)):
    words.append(str(df1.iloc[ii]['text']).split(" "))    
    
n_gram_all = []

for word in words:
    # get n-grams for the instance
    n_gram = []
    for i in range(len(word)-2+1):
        n_gram.append("".join(word[i:i+2]))
    n_gram_all.append(n_gram)

--> for i in range(len(word)-2+1): inner loop that generates n-grams for the current word. It iterates over the range from 0 to the length of the word minus 2 plus 1. This range allows for generating n-grams of length 2.

--> n_gram.append("".join(word[i:i+2])): For each iteration of the inner loop, it retrieves a substring of length 2 from the current word and joins them together to form an n-gram. The resulting n-gram is added to the n_gram list.

After generating all the n-grams for the current word, the n_gram list is added to the n_gram_all list which contains the generated n-grams for each word in the text data. The n-grams are formed by concatenating adjacent pairs of words within each instance.

In [ ]:
from sklearn.feature_extraction.text import HashingVectorizer

# hash vectorizer instance
hvec = HashingVectorizer(lowercase=False, analyzer=lambda l:l, n_features=2**12)

# features matrix X
X = hvec.fit_transform(n_gram_all)

first HashingVectorizer class will be imported. It is then used to create an instance of the HashingVectorizer class.
this class provides functionality for converting text data into a fixed-size vector representation using the hashing trick.

lowercase=False --> This parameter indicates that the text should not be converted to lowercase. By default, HashingVectorizer converts text to lowercase.

analyzer=lambda l:l --> lambda function that simply returns the input as it is, without any further preprocessing or tokenization, that is to ensure that the input n-grams are treated as individual tokens.

n_features=2**12 --> This parameter sets the number of features or dimensions in the resulting vector representation, here 2^12 (4096) features are used. The choice of the number of features depends on the specific application and available computational resources.

In [ ]:
X.shape

In [ ]:
from sklearn.model_selection import train_test_split

# test set size of 20% of the data and the random seed 42 <3
X_train, X_test = train_test_split(X.toarray(), test_size=0.2, random_state=42)

print("X_train size:", len(X_train))
print("X_test size:", len(X_test), "\n")

**K-means clustering for 15 cluster**

In [ ]:
from sklearn.cluster import KMeans

k = 15 
kmeans = KMeans(n_clusters=k, n_jobs=4, verbose= k)
y_pred = kmeans.fit_predict(X_train)

n_jobs=4: This parameter specifies the number of parallel jobs to run for the computation. It is set to 4, indicating that the computation can be distributed across 4 CPU cores if available.

verbose=k: This parameter controls the verbosity of the algorithm. It determines the amount of output text during the clustering process. In this case, the value of k is used, so the algorithm will provide more detailed output.

In [ ]:
y_pred.shape

In [ ]:
y_train = y_pred
y_test = kmeans.predict(X_test)

In [ ]:
%%time
from sklearn.manifold import TSNE

tsne = TSNE(verbose=1)
X_embedded = tsne.fit_transform(X_train)

In [ ]:
from matplotlib import pyplot as plt
import seaborn as sns

# sns settings
sns.set(rc={'figure.figsize':(10,10)})

# colors
palette = sns.color_palette("bright", 1)

# plot
sns.scatterplot(X_embedded[:,0], X_embedded[:,1], palette=palette)

plt.title("t-SNE Covid-19 Articles")
# plt.savefig("plots/t-sne_covid19.png")
plt.show()

# Visualizing clusters

In [ ]:
X_embedded[:,1].shape

In [ ]:
# sns settings
sns.set(rc={'figure.figsize':(10,10)})

# colors
palette = sns.color_palette("bright", len(set(y_pred)))

# plot
sns.scatterplot(X_embedded[:,0], X_embedded[:,1], hue=y_pred, legend='full', palette=palette)
plt.title("t-SNE Covid-19 Articles - Clustered")
# plt.savefig("plots/t-sne_covid19_label.png")
plt.show()

# Part 3 : working with clean_noncomm_use.csv

In [ ]:
type(clean_noncomm_use.abstract.dropna().tolist())

gensim.summarization offers TextRank summarization

In [ ]:
from gensim.summarization.summarizer import summarize
summarize(clean_noncomm_use.abstract.dropna().to_string())

In [ ]:
import numpy as np
import pandas as pd
import nltk
nltk.download('punkt') # one time execution
import re

#  Understanding the Text Rank Algorithm

# TextRank Algorithm

**We will apply the TextRank algorithm on this dataset of  articles with the aim of creating a concise summary.**

Now the next step is to break the text into individual sentences. We will use the sent_tokenize( ) function of the nltk library to do this.

In [ ]:
#ref : https://www.analyticsvidhya.com/blog/2018/11/introduction-text-summarization-textrank-python/
from nltk.tokenize import sent_tokenize
sentences = []
for s in clean_noncomm_use.abstract.dropna():
    sentences.append(sent_tokenize(s))

sentences = [y for x in sentences for y in x] # flatten list

In [ ]:
sentences[:3]

# Part 4 : Analyzing clean_pmc.csv file

In [ ]:
!pip install -U transformers
!pip install -U torch

In [ ]:
import torch
import os
import json
from transformers import T5Tokenizer, T5ConditionalGeneration, T5Config

# Summarization Task using T5 model

In [ ]:
df1 = biorxiv_clean
df1 = df1.abstract.dropna()
df1abstracts = df.values.tolist()

len(df1abstracts)

In [ ]:
T5_PATH = 't5-base'
t5_model = T5ForConditionalGeneration.from_pretrained(T5_PATH, output_past=True)
t5_tokenizer = T5Tokenizer.from_pretrained(T5_PATH)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
def t5_summarize(input_text, num_beams=4, num_words=80):
    #input_text = str(input_text).replace('\n', '')
    input_text = ' '.join(input_text.split())
    input_tokenized = t5_tokenizer.encode(input_text, return_tensors="pt").to(device)
    summary_task = torch.tensor([[21603, 10]]).to(device)
    input_tokenized = torch.cat([summary_task, input_tokenized], dim=-1).to(device)
    summary_ids = t5_model.generate(input_tokenized,
                                    num_beams=int(num_beams),
                                    no_repeat_ngram_size=3,
                                    length_penalty=2.0,
                                    min_length=30,
                                    max_length=int(num_words),
                                    early_stopping=True)
    output = [t5_tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) 
              for g in summary_ids]
    return output[0]

In [ ]:
%%time
for i in range(20):
    try:
        print('BioArvix paper  ',i + 1, " : \n" )
        print(t5_summarize(df1abstracts[i]))
        print('............................................................................\n\n\n\n')
    except:
        print('paper ',i+1 ," has LONG ABSTRACT\n\n")

# **evaluation metric** #

In [ ]:
!pip install evaluate
!pip install rouge-score
!pip install rouge

In [ ]:
from rouge import Rouge

# Define the generated summary and reference summary
generated_summary = '''lowering the age threshold for quarantine to 50 years of age ICU 
admissions drastically.similer results are expected to hold for other countries, though 
some minor adaption will be required'''
reference_summary = '''Lowering the age threshold for quarantine to 50 years of age could 
lead to a significant reduction in ICU admissions. Similar findings are anticipated for other
countries,although slight modifications may be necessary.'''

# Initialize Rouge scorer
rouge_scorer = Rouge()

# Calculate ROUGE scores
rouge_scores = rouge_scorer.get_scores(generated_summary, reference_summary)

# Print ROUGE scores
print("ROUGE Scores:")
print(rouge_scores)
